In [2]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [ ]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [ ]:
tables = soup.find_all('table',class_='wikitable sortable')

In [ ]:
len(tables)

In [ ]:
type(tables[0])

In [ ]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [ ]:
dfs = [df1, df2, df3, df4]
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df

In [ ]:
df_2020 = df[['Title','Cast and crew']]

In [ ]:
df_2020

In [ ]:
!pip install tmdbv3api

In [ ]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '8ac87b8a4e208304fe864743938c083c'

In [ ]:
def get_genre(title):
    api_key = '8ac87b8a4e208304fe864743938c083c'  
    search_url = 'https://api.themoviedb.org/3/search/movie'
    movie_params = {
        'api_key': api_key,
        'query': title
    }
    response = requests.get(search_url, params=movie_params)
    data = response.json()
    if data['results']:
        movie_id = data['results'][0]['id']
        movie_info_url = f'https://api.themoviedb.org/3/movie/{movie_id}'
        movie_info_params = {
            'api_key': api_key
        }
        response = requests.get(movie_info_url, params=movie_info_params)
        movie_data = response.json()
        if movie_data.get('genres'):
            return ', '.join(genre['name'] for genre in movie_data['genres'])
    return np.nan

data = {
    'Title': ['The Grudge', 'Underwater', 'Like a Boss', 'Three Christs'],
    'Cast and crew': ['Nicolas Pesce (director/screenplay); Andrea Ri...', 'William Eubank (director); Brian Duffield, Ada...', 'Miguel Arteta (director); Sam Pitman, Adam Col...', 'Jon Avnet (director/screenplay); Eric Nazarian...']
}
df_2020 = pd.DataFrame(data)

# Apply the get_genre function to each title in the DataFrame
df_2020['genres'] = df_2020['Title'].apply(get_genre)

# Display the DataFrame with genres
print(df_2020)

In [ ]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2020

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2020

In [ ]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [ ]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df20

In [ ]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20 = new_df20.dropna(how='any')

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [ ]:
new_df20

In [ ]:
old_df = pd.read_csv('final_data.csv')

In [ ]:
old_df

In [ ]:
final_dfs = [old_df, new_df20]
final_df = pd.concat(final_dfs,ignore_index=True)

In [ ]:
final_df

In [ ]:
final_df.to_csv('main_data.csv',index=False)